<a href="https://colab.research.google.com/github/Hasibul-Islam/bicycl_detection_yolov10/blob/main/bicycle_detection_yolov10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# dataset labeling and splitting

import os
import xml.etree.ElementTree as ET
import shutil
import random

# ======== CONFIG ========
dataset_folder = "/content/gdrive/MyDrive/bicycle_raw"  # your folder with jpg + xml
output_folder = "/content/gdrive/MyDrive/bicycle_dataset"       # new YOLO dataset folder
split_ratio = 0.8  # 80% train, 20% val
class_name_to_id = {"Bicycle": 0}  # class mapping
# ========================

# Create folders
for split in ['train', 'val']:
    os.makedirs(os.path.join(output_folder, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'labels', split), exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(dataset_folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
random.shuffle(image_files)

# Split into train/val
split_index = int(len(image_files) * split_ratio)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

def convert_xml_to_yolo(xml_path, img_w, img_h):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    yolo_lines = []

    for obj in root.findall('object'):

        cls_name = obj.find('name').text
        if cls_name not in class_name_to_id:
            continue
        cls_id = class_name_to_id[cls_name]

        xml_box = obj.find('bndbox')
        xmin = float(xml_box.find('xmin').text)
        ymin = float(xml_box.find('ymin').text)
        xmax = float(xml_box.find('xmax').text)
        ymax = float(xml_box.find('ymax').text)

        # Convert to YOLO format
        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h

        yolo_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    return yolo_lines

def process_split(files_list, split_name):

    for img_file in files_list:


        xml_file = os.path.splitext(img_file)[0] + ".xml"
        img_path = os.path.join(dataset_folder, img_file)
        xml_path = os.path.join(dataset_folder, xml_file)

        # Copy image
        shutil.copy(img_path, os.path.join(output_folder, 'images', split_name, img_file))

        # Get image size from XML
        tree = ET.parse(xml_path)
        root = tree.getroot()
        img_w = int(root.find('size').find('width').text)
        img_h = int(root.find('size').find('height').text)

        # Convert and save YOLO label
        yolo_data = convert_xml_to_yolo(xml_path, img_w, img_h)
        label_path = os.path.join(output_folder, 'labels', split_name, os.path.splitext(img_file)[0] + ".txt")
        with open(label_path, 'w') as f:
            f.write("\n".join(yolo_data))

# Process train and val
process_split(train_files, 'train')
process_split(val_files, 'val')

print("✅ Dataset prepared in YOLO format at:", output_folder)


✅ Dataset prepared in YOLO format at: /content/drive/MyDrive/bicycle_dataset


In [25]:


# =====================
# STEP 2: Install YOLOv10 (Ultralytics)
# =====================
!pip install ultralytics opencv-python --quiet

# =====================
# STEP 3: Create data.yaml
# =====================
yaml_content = """
path: /content/drive/MyDrive/bicycle_dataset
train: images/train
val: images/val
names:
  0: Bicycle
"""
#with open("/content/drive/MyDrive/bicycle_dataset/data.yaml", "w") as f:
    #f.write(yaml_content)

#print("✅ data.yaml created at /content/drive/MyDrive/bicycle_dataset/data.yaml")

# =====================
# STEP 4: Train YOLOv10
# =====================
from ultralytics import YOLO

# Load YOLOv10n (nano, fastest) — change to yolov10s/m/l/x for bigger models
model = YOLO("yolov10n.pt")

# Train
model.train(
    data= "/content/gdrive/MyDrive/bicycle_dataset/data.yaml",
    epochs=50,            # adjust based on dataset size & GPU
    imgsz=640,            # image size
    batch=16,             # adjust to fit your GPU memory
    project="/content/gdrive/MyDrive/yolov10_train",
    name="bicycle_model"
)

print("✅ Training complete!")




Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gdrive/MyDrive/bicycle_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=bicycle_model2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, pl

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1      9856  ultralytics.nn.modules.block.SCDown          [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1     36096  ultralytics

 23        [16, 19, 22]  1    861718  ultralytics.nn.modules.head.v10Detect        [1, [64, 128, 256]]           
YOLOv10n summary: 223 layers, 2,707,430 parameters, 2,707,414 gradients, 8.4 GFLOPs

Transferred 493/595 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.4±0.1 ms, read: 220.3±300.5 MB/s, size: 3300.0 KB)


train: Scanning /content/gdrive/MyDrive/bicycle_dataset/labels/train... 158 images, 0 backgrounds, 0 corrupt: 100%|██████████| 158/158 [02:13<00:00,  1.19it/s]

...

      50/50      4.03G      1.069      1.663      1.948         15        640: 100%|██████████| 10/10 [00:05<00:00,  1.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.14it/s]

                   all         40         44      0.951      0.932      0.946      0.711



50 epochs completed in 0.152 hours.
Optimizer stripped from /content/gdrive/MyDrive/yolov10_train/bicycle_model2/weights/last.pt, 5.7MB
Optimizer stripped from /content/gdrive/MyDrive/yolov10_train/bicycle_model2/weights/best.pt, 5.7MB

Validating /content/gdrive/MyDrive/yolov10_train/bicycle_model2/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10n summary (fused): 102 layers, 2,265,363 parameters, 0 gradients, 6.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.35it/s]


                   all         40         44      0.952      0.905      0.948      0.732
Speed: 0.2ms preprocess, 5.4ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /content/gdrive/MyDrive/yolov10_train/bicycle_model2
✅ Training complete!


In [7]:
import cv2
import time
import shutil
import os
from ultralytics import YOLO

# Step 1: Copy video to local storage
drive_video_path = "/content/gdrive/MyDrive/bicycle_dataset/bicycle.mp4"
local_video_path = '/content/video.mp4'

if not os.path.exists(local_video_path):
    print("Copying video from Google Drive to local storage...")
    shutil.copy(drive_video_path, local_video_path)
    print("Copy complete!")
else:
    print("Video already exists locally.")

# Step 2: Load YOLOv10 model
#model_path = "/content/gdrive/MyDrive/yolov10_train/bicycle_model2/weights/best.pt"
#model = YOLO(model_path)
model = YOLO('yolov10n.pt')

# Step 3: Open video
cap = cv2.VideoCapture(local_video_path)
if not cap.isOpened():
    raise RuntimeError("Failed to open video!")

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Save locally first
output_path = "/content/bike_detected.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print(f"Processing video: {frame_count} frames at {fps} FPS...")

# Step 4: Process frames
frame_idx = 0
while True:
    t0 = time.time()
    ret, frame = cap.read()
    t1 = time.time()

    if not ret:
        break

    # Run YOLO inference and draw boxes
    results = model(frame)
    frame_with_boxes = results[0].plot()  # Draw boxes

    # Write frame to output video
    out.write(frame_with_boxes)

    print(f"Frame {frame_idx}: Read {(t1-t0)*1000:.1f} ms | Inference {(time.time()-t1)*1000:.1f} ms")
    frame_idx += 1

cap.release()
out.release()

# Step 5: Copy video to Google Drive
shutil.copy(output_path, "/content/gdrive/MyDrive/bicycle_dataset/bike_detected.mp4")
print("Processing complete! Video saved to Google Drive!")


Video already exists locally.


Streaming output truncated to the last 5000 lines.
0: 384x640 1 person, 6 bicycles, 2 cell phones, 120.1ms
Speed: 5.6ms preprocess, 120.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Frame 174: Read 3.6 ms | Inference 158.0 ms

0: 384x640 1 person, 6 bicycles, 1 cell phone, 121.8ms
Speed: 7.5ms preprocess, 121.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Frame 175: Read 3.6 ms | Inference 164.7 ms

0: 384x640 1 person, 6 bicycles, 1 cell phone, 122.0ms
Speed: 6.0ms preprocess, 122.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Frame 176: Read 3.6 ms | Inference 163.1 ms

0: 384x640 1 person, 6 bicycles, 1 cell phone, 138.1ms
Speed: 7.6ms preprocess, 138.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Frame 177: Read 3.7 ms | Inference 173.8 ms

0: 384x640 1 person, 6 bicycles, 1 cell phone, 121.0ms
Speed: 6.3ms preprocess, 121.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Fra

In [ ]:
# =====================
# STEP 5: Run Detection on Video
# =====================
# Path to your trained model



model_path = "/content/gdrive/MyDrive/yolov10_train/bicycle_model2/weights/best.pt"

# Path to input video in Drive
video_path = "/content/gdrive/MyDrive/bicycle_dataset/bicycle.mp4"  # <-- Change to your video path



# Output path
output_path = "/content/gdrive/MyDrive/bicycle_dataset/bike_detected.mp4"

# Run detection
model = YOLO(model_path)
model.predict(
    source=video_path,
    conf=0.25,
    save=True,
    project="/content/gdrive/MyDrive/bicycle_dataset",
    name="bike_detected"
)

print(f"✅ Detection complete! Check output folder: /content/gdrive/MyDrive/bicycle_dataset/bike_detected")


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1424) /content/gdrive/MyDrive/bicycle_dataset/bicycle.mp4: 384x640 1 Bicycle, 10.0ms
video 1/1 (frame 2/1424) /content/gdrive/MyDrive/bicycle_dataset/bicycle.mp4: 384x640 1 Bicycle, 12.3ms
video 1/1 (frame 3/1424) /content/gdrive/MyDrive/bicycle_dataset/bicycle.mp4: 384x640 1 Bicycle, 22.2ms
video 1/1 (frame 4/1424) /content/gdrive/MyDrive/bicycle_dataset/bicycle.mp4: 384x640 1 Bicycle, 10.5ms
video 1/1 (frame 5/1424) /content/gdrive

In [19]:
from google.colab import drive

drive.flush_and_unmount()  # unmounts Google Drive




Drive not mounted, so nothing to flush and unmount.


In [16]:
import os

file_path = "/content/drive/MyDrive/bicycle_raw/"

if os.path.exists(file_path):
    print("File exists")
else:
    print("File does not exist")
print(os.getcwd())

print(os.listdir("/content/drive/MyDrive"))

File does not exist
/content
['yolov10_train']
